<img src='images/organizations-logos.png' align='center' width='100%'></img>

# Data Assimilation Practical · Joint Training in Atmospheric Composition (2021)

This practical exercise was created for the <a href ="https://atmosphere.copernicus.eu/3rd-eumetsatesaecmwf-joint-training-atmospheric-composition" target = "_blank">3rd EUMETSAT/ESA/ECMWF Joint Training in Atmospheric Composition (6-17 December, 2021)</a> to show the assimilation of NO<sub>2</sub> observations from the TROPOspheric Monitoring Instrument (TROPOMI) aboard Sentinel 5-P into the forecasts of the Copernicus Atmosphere Monitoring Service (CAMS). It is divided into the following sections:

1. [Installation](#installation): A brief guide to know how to install the <a href = "https://github.com/esowc/adc-toolbox/" target = "_blank">Atmospheric Datasets Comparison (ADC) Toolbox</a>, which contains functions that facilitate the datasets retrieval, metadata merge and statistical analysis.

2. [Datasets retrieval](#datasets_retrieval): The model and sensor datasets are downloaded and read as xarray objects before assimilating the real observations into the model dataset.

3. [Units conversion](#units_conversion): The units of both datasets are converted to molecules/cm<sup>2</sup>.

4. [Data assimilation](#data_assimilation): The model partial columns are interpolated into the TM5 grid and the averaging kernels are applied.

5. [Comparison analysis](#comparison_analysis): Statistical methods are used to better understand the differences between both datasets and the effects of the data assimilation process.

## <a id='installation'></a>1. Installation

### Clone the repository and set up the virtual environment

Participants should <a href = "https://my.wekeo.eu/web/guest/user-registration" target = "_blank">create an account in Wekeo</a> to use the JupyterHub and run this notebook. Once they <a href = "https://jupyterhub-wekeo.apps.eumetsat.dpi.wekeo.eu" target = "_blank">have access to this service</a>, they can open the terminal, or an empty Jupyter Notebook, and clone the ADC Toolbox repository with the command:

```bash
$ git clone https://github.com/esowc/adc-toolbox
```

The virtual environment <em>environment.yml</em> was generated to simplify the installation process, so you just need to create this environment from the file and activate it to install the necessary libraries and packages. Since this process might take up some time, it is better if the users simulate it by:

```bash
$ conda create --name adc-toolbox
$ conda activate adc-toolbox
$ conda install -c conda-forge/label/cartopy_dev cartopy
$ pip install -r requirements.txt
$ python -m ipykernel install --user --name adc-toolbox
```

After running the previous commands, the page should be refreshed and the correct kernel (adc-toolbox) should be selected.

To finalize the installation process, users need to create a text file under the training data folder, with the name <em>keys.txt</em>, and write down their personal CAMS API key in one line with the format <em>UID:Key</em>. This key can be obtained by <a href = "https://ads.atmosphere.copernicus.eu/user/register?">registering at the Atmosphere Data Store</a>.

### Import libraries

In [ ]:
# Related to the system
import os 
from pathlib import Path

# Related to the data retrieval
from sentinelsat.sentinel import SentinelAPI, geojson_to_wkt
import cdsapi
import cfgrib
import geojson
import urllib3

# Related to the data analysis
import math
import xarray as xr
import pandas as pd
import numpy as np
import datetime as dt
from itertools import product
import scipy.interpolate
from sklearn.linear_model import LinearRegression
from scipy.spatial.distance import cdist

# Related to the results
from copy import copy
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature
import geocoder
import seaborn as sns
from matplotlib import animation
from IPython.display import HTML, display

### Import functions

In [ ]:
%run ../../functions/functions_general.ipynb
%run ../../functions/functions_cams.ipynb
%run ../../functions/functions_tropomi.ipynb

### Settings

In [ ]:
# Hide pandas warning
pd.options.mode.chained_assignment = None

# Hide API request warning
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Increase animation limit
matplotlib.rcParams['animation.embed_limit'] = 25000000

## <a id='datasets_retrieval'></a>2. Datasets retrieval

### Available datasets

ADC Toolbox facilitates the data retrieval of all the datasets presented in Table 1, since the dates they became available to the public. As an exception, the retrieval of IASI L2 data is currently available only since May 14, 2019.

<p align="center"> Table 1. Temporal availability (start date - present) by data source.</p>

| Dataset | Type | NO<sub>2</sub> | O<sub>3</sub> | CO | SO<sub>2</sub> | HCHO |
| :---: | :---: | :---: | :---: | :---: | :---: | :---: |
| CAMS  | <a href = "https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/CAMS-global-atmospheric-composition-forecasts" target = "_blank">Forecast</a> | 01.2015 | 01.2015 | 01.2015 | 01.2015 | 01.2015 | 
| CAMS  | <a href = "https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/CAMS-global-ghg-reanalysis-egg4-monthly">Reanalysis</a> | 01.2003 | 01.2003 | 01.2003 | 01.2003 | 01.2003 | 
| TROPOMI  | <a href = "https://s5phub.copernicus.eu/dhus/" target = "_blank">L2</a> | 07.2018 | 07.2018 | 07.2018 | 10.2018 | - | 
| IASI  | <a href = "https://iasi.aeris-data.fr/" target = "_blank">L2</a> | - | 01.2008 | 10.2007 | 10.2007 | - |
| IASI  | <a href = "https://iasi.aeris-data.fr/" target = "_blank">L3</a> | - | 01.2008 | 10.2007 | 10.2007 | - |
| GOME-2  | <a href = "https://acsaf.org/offline_access.php" target = "_blank">L2</a> | 01.2007 | 01.2007 | - | 01.2007 | 01.2007 | 
| GOME-2  | <a href = "https://acsaf.org/offline_access.php" target = "_blank">L3</a> | 02.2007 | 01.2007 | - | 01.2007 | 01.2007 | 

Sentinel 5-P was launched in 2017 and, as it can be read in the table, the L2 products are accessible since:

* July 2018 for NO<sub>2</sub>, O<sub>3</sub> and CO concentrations.
* October 2018 for SO<sub>2</sub> concentrations.

In order to automatically download any model or sensor dataset, users only need to define:

* Name of the atmospheric component.
* Short (<em>cams</em>) and full name of the model (<em>cams-global-atmospheric-composition-forecasts</em> or <em>cams-global-reanalysis-eac4-monthly</em>).
* Short name of the sensor (<em>tropomi</em>, <em>iasi</em> or <em>gome</em>).
* Start and end date: a range will be calculated from both dates.
* Bounding box by coordinates.

In [ ]:
# Define component
component_nom = 'NO2'

# Define model
model = 'cams'
model_full_name = 'cams-global-atmospheric-composition-forecasts'

# Define sensor
sensor = 'tropomi'
sensor_type = 'L2'
apply_kernels = True

# Define search period
start_date = '2021-10-28'
end_date = '2021-10-28'

# Define extent
lon_min = 12
lon_max = 18.9
lat_min = 48.5
lat_max = 51.1

### Comparison checker and folder generation

The toolbox will check if the merge and comparison between the specified model and sensor is possible, given the name of the species. If it is, the molecular weight and some metadata will be obtained. Afterwards, it will create the folders where the datasets will be stored.

This notebook can only be used to carry out the data assimilation process, please refer to the main code in case you wish to compare the CAMS model against the observations from IASI or GOME-2.

In [ ]:
# Check if comparison is possible
comparison_check(sensor, model, component_nom, model_full_name, sensor_type)

# Get component full name and molecular weight
component, component_mol_weight, product_type, sensor_column = components_table(sensor, component_nom, sensor_type)

# Folders generation
generate_folders(model, sensor, component_nom, sensor_type)

### Search period and bounding box

The search period and bounding box are derived from the details that were provided in advance.

In [ ]:
# Generate array with search dates
dates = search_period(start_date, end_date, sensor, sensor_type)

# Create bbox
bbox = search_bbox(lon_min, lat_min, lon_max, lat_max)

### Download and read the model data

The model dataset will be downloaded as a GRIB file and read as a xarray object. In this step, the users can decide if they want to retrieve total (<em>model_level = 'Single'</em>) or partial columns (<em>model_level = 'Multiple'</em>). In the data assimilation process, we need to obtain the partial columns and, particularly in this case, at 137 vertical levels.

In [ ]:
model_product_name, model_type = CAMS_download(dates, start_date, end_date, component, 
                                               component_nom, model_full_name, model_level = 'Multiple')

In [ ]:
model_ds, _ = CAMS_read(model_product_name, component, component_nom, dates)
model_ds

### Download and read sensor data

The sensor dataset will be downloaded as a NetCDF file and read as a xarray object, along with its details in the attached datasets. More information about the product can be found in the <a href = "http://www.tropomi.eu/sites/default/files/files/S5P-KNMI-L2-0021-MA-Product_User_Manual_for_the_Sentinel_5_precursor_Nitrogen_dioxide-0.8.1_20151207_signed.pdf" target = "_blank">TROPOMI NO<sub>2</sub> product manual</a>.

In [ ]:
dates = sensor_download(sensor, sensor_type, component_nom, dates, bbox, product_type)

In [ ]:
sensor_ds, support_input_ds, support_details_ds = sensor_read(sensor, sensor_type, sensor_column, 
                                                              component_nom, dates)

In [ ]:
sensor_ds

Within <em>support_input_ds</em> we can find the surface pressure data that we need to compute the pressure at each level, while <em>support_details_ds</em> contains the processing quality flags and air mass factors to calculate the column kernels.

In [ ]:
support_input_ds

In [ ]:
support_details_ds

## <a id='units-conversion'></a>3. Units conversion

### Convert the model data units (from kg/kg to molecules/cm<sup>2</sup>)

#### Retrieve auxiliary data

The details of the 137 vertical levels in the L137 model are defined by ECMWF and are necessary to evaluate the levels pressure. From the information given, we will particularly need the coefficients <em>a</em> and <em>b</em>.


In [ ]:
model_levels_df = CAMS_137_levels()
model_levels_df

#### Calculate the columns above each half level (kg/kg to kg/m<sup>2</sup>)

To convert the original units (kg/kg) into molecules/cm<sup>2</sup>, we need to do an intermediate step: calculating the NO<sub>2</sub> columns above each CAMS half level assuming that they are 0 at the top of the atmosphere, converting the units to kg/m<sup>2</sup>.

In [ ]:
# Calculate level pressures from the surface pressures
model_ds = CAMS_pressure(model_ds, model_levels_df, start_date, end_date, component_nom)

In [ ]:
print('The columns above each model half level will be calculated.')

# Initialize new array
model_ds_all = []

for time in model_ds.time:

    PC_hybrid = []
    
    model_ds_time_old = model_ds.sel(time = time)

    # Initialize partial columns at the top of the atmosphere as 0
    PC_hybrid_0 = model_ds_time_old.sel(hybrid = 1)
    PC_hybrid_0['component'] = PC_hybrid_0['component'].where(PC_hybrid_0['component'] <= 0, 0, drop = False)
    PC_hybrid_0 = PC_hybrid_0.expand_dims(dim = ['hybrid'])
    PC_hybrid.append(PC_hybrid_0)
    model_ds_time_new = PC_hybrid_0

    for hybrid in range(1, 136):

        # Calculate partial columns above each model level
        PC_last = model_ds_time_new.component.sel(hybrid = hybrid)
        PC_current = model_ds_time_old.component.sel(hybrid = hybrid + 1)
        pressure_last = model_ds_time_old.pressure.sel(hybrid = hybrid)
        pressure_current = model_ds_time_old.pressure.sel(hybrid = hybrid + 1)
        pressure_diff = pressure_current - pressure_last

        # Units: (kg/kg * kg/m*s2) * s2/m -> kg/m2
        PC_above = model_ds_time_old.sel(hybrid = hybrid + 1)
        PC_above['component'] = PC_last + PC_current * pressure_diff * (1/9.81)
        PC_hybrid.append(PC_above)
        model_ds_time_new = xr.concat(PC_hybrid, pd.Index(range(1, hybrid + 2), name = 'hybrid'))

    model_ds_all.append(model_ds_time_new)

model_ds = xr.concat(model_ds_all, dim = 'time')

# Assign new units to array
units = 'kg m**-2'
model_ds['component'] = model_ds.component.assign_attrs({'units': units})
print('The model component units have been converted from kg kg**-1 to kg m**-2.')

#### Convert units with Avogadro's number (kg/m<sup>2</sup> to molecules/cm<sup>2</sup>)

After, we convert the data units from kg/m<sup>2</sup> to molecules/cm<sup>2</sup> simply by:

In [ ]:
# Get Avogadro's number
NA = 6.022*10**23
model_ds['component'] = (model_ds['component'] * NA * 1000) / (10000 * component_mol_weight)

# Assign new units to array
units = 'molec cm-2'
model_ds['component'] = model_ds.component.assign_attrs({'units': units})
print('The model component units have been converted from kg m**-2 to molec cm-2.')

In [ ]:
model_ds

### Convert TROPOMI data units (From mol/m<sup>2</sup> to molecules/cm<sup>2</sup>)

In [ ]:
sensor_ds['sensor_column'] = sensor_ds['sensor_column'] * 6.02214*10**19
sensor_ds['sensor_column'] = sensor_ds['sensor_column'].assign_attrs({'units': 'molec cm-2'})

print('The sensor component units have been converted from mol cm-2 to molec cm-2.')

In [ ]:
sensor_ds

## <a id='data_assimilation'></a>4. Data assimilation

In [ ]:
match_table = generate_match_table(sensor_ds, model_ds, bbox,  
                                   sensor, component_nom, apply_kernels)
match_table

In [ ]:
merge_table = generate_merge_table(match_table, sensor_ds, model_ds, sensor, apply_kernels)
merge_table

In [ ]:
descr_statistics_table = merge_table.describe()
descr_statistics_table

## <a id='comparison_analysis'></a>5. Comparison analysis

### Select plot dates

In [ ]:
plot_dates = plot_period(sensor_ds, sensor)

### Select plot extent

In [ ]:
plot_bbox = plot_extent(bbox)

### Compare model and TROPOMI total columns

In [ ]:
# Choose distribution (aggregated, individual or animated)
distribution_type = 'individual'

# Choose range (original, equal or manual)
range_type = 'equal'
vmin_manual = None
vmax_manual = None

# Define projection and colors
projection = ccrs.PlateCarree()
color_scale = 'coolwarm' 

visualize_model_vs_sensor(model, sensor, component_nom, units, merge_table, plot_dates, plot_bbox, 20, 0.80, 
                          model_type, sensor_type, range_type, distribution_type, projection,
                          color_scale, vmin_manual, vmax_manual)

### Retrieve nearest values to specific latitude and longitude

In [ ]:
coords_search_list = (50, 60,
                      4, 10,
                      20, 30)
retrieval_table_all =  retrieve_coords(merge_table.dropna(), coords_search_list, component_nom, 
                                       sensor, model, plot_dates, units)
retrieval_table_all

### Scatter plots by bbox

In [ ]:
show_seasons = False
extent_definition = 'bbox' # bbox or country
scatter_plot_type = 'individual' # aggregated or individual

lim_min = None
lim_max = None

summary = scatter_plot(merge_table.dropna(), component_nom, units, sensor, plot_dates, 1.05, 
                       extent_definition, show_seasons, scatter_plot_type, lim_min, lim_max, plot_bbox)

In [ ]:
summary

### Scatter plots by season

In [ ]:
show_seasons = True
extent_definition = 'bbox' # bbox or country
scatter_plot_type = 'individual' # aggregated or individual

lim_min = None
lim_max = None

summary = scatter_plot(merge_table.dropna(), component_nom, units, sensor, plot_dates, 1.05, 
                       extent_definition, show_seasons, scatter_plot_type, lim_min, lim_max, plot_bbox)

In [ ]:
summary

### Scatter plots by country (Google API required!)

In [ ]:
"""
show_seasons = False
extent_definition = 'country' # bbox or country
scatter_plot_type = 'aggregated' # aggregated or individual
plot_countries = ['Czech Republic', 'Poland', 'Germany']

lim_min = None
lim_max = None

scatter_plot(merge_table.dropna(), component_nom, units, sensor, plot_dates, 1.05, 
             extent_definition, show_seasons, scatter_plot_type, lim_min, lim_max, plot_countries)
"""

In [ ]:
#summary